In [1]:
import io
import numpy as np
import pickle

In [2]:
data_path = '../data'

In [3]:
# Process fasttext embeddings
# https://fasttext.cc/docs/en/english-vectors.html
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

In [4]:
# English
ft_embed_en = load_vectors(os.path.join(data_path, 'wiki-news-300d-1M-subword.vec'))
len(ft_embed_en)

999994

In [5]:
# French
ft_embed_fr = load_vectors(os.path.join(data_path, 'cc.fr.300.vec'))
len(ft_embed_fr)

2000000

In [17]:
# Load the vocabulary list from the training data
data = pickle.load(open(os.path.join(data_path, 'translate-data.pkl'), 'rb'))
X_itos, y_itos = data['X_itos'], data['y_itos']
len(X_itos), len(y_itos)

(12633, 21655)

In [10]:
# Map words to pre-trained embeddings
def map_word_to_pretrained_embedding(embed_index, embed_size, toks_itos):
    vocab_size = len(toks_itos)
    embed_matrix = np.zeros((vocab_size, embed_size))
    
    # Keep a running count of unmatched words
    not_found = []
    
    for i, word in enumerate(toks_itos):
        if word in embed_index:
            embed_vec = embed_index[word]
            embed_matrix[i] = embed_vec
        else:
            not_found.append(word)

    return embed_matrix, not_found

In [18]:
# English
X_ft_embed, X_not_found = map_word_to_pretrained_embedding(ft_embed_en, 300, X_itos)
X_ft_embed.shape, 1.0 * len(X_not_found) / len(X_itos)

((12633, 300), 0.011557033167102035)

In [19]:
# French
y_ft_embed, y_not_found = map_word_to_pretrained_embedding(ft_embed_fr, 300, y_itos)
y_ft_embed.shape, 1.0 * len(y_not_found) / len(y_itos)

((21655, 300), 0.02202724543985223)

In [22]:
# Save
pickle.dump(X_ft_embed, open(os.path.join(data_path, 'X_ft_embed.pkl'), 'wb'))
pickle.dump(y_ft_embed, open(os.path.join(data_path, 'y_ft_embed.pkl'), 'wb'))